# FashionMNIST Dataset & DataLoader 구성

In [30]:
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor


In [31]:
# FashionMNIST 데이터세트 다운로드

train_data = datasets.FashionMNIST(
    root="data", 
    train=True, 
    download=True, 
    transform=ToTensor() 
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [32]:
train_dataloader = DataLoader(train_data, batch_size=64, shuffle=True)

test_dataloader  = DataLoader(test_data, batch_size=64, shuffle=False)

# 모델링

In [33]:
from torch import nn

class NeuralNetwork(nn.Module):

    def __init__(self):
        # nn.Module 생성
        super(NeuralNetwork, self).__init__() 
        
        # 레이어 정의
        self.flatten = nn.Flatten() # 평탄화 레이어 정의
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        # 입력되는 x의 모양이 (64, 1, 28, 28) -> (N, C, H, W)
            # 입력 이미지에 대한 평탄화가 필요
        x = self.flatten(x) # flatten 레이어를 지나게 되면 (64, 784)
        y = self.linear_relu_stack(x)

        return y

# 모델 생성

In [34]:
# 파이토치를 이용해 모델 객체를 만들고 나서 어떤 장치(device) 환경에서 훈련이나 추론을 수행할지 결정지어주기

import torch

# MPS 지원 여부 확인
if torch.backends.mps.is_available():
    device = 'mps'  # Apple Silicon에서 MPS 사용
elif torch.cuda.is_available():
    device = 'cuda'  # CUDA 사용 가능 시
else:
    device = 'cpu'  # 그 외에는 CPU 사용

print(f"Using device: {device}")


Using device: mps


In [35]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=10, bias=True)
  )
)


# 모델 훈련

In [36]:
loss_fn = nn.CrossEntropyLoss() # 이미 여기에 소프트맥스 함수가 포함되어 있다.
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [37]:
# 훈련 과정
    # 데이터 로딩 -> 예측 -> Loss 계산 -> 미분(backward) -> 최적화

def train_loop(dataloader, model, loss_fn, optimizer):
    
    size = len(dataloader.dataset)

    # 모델을 훈련 모드로 설정
    model.train()

    for batch, (X, y) in enumerate(dataloader):
        
        # 데이터 로더에 들어있던 텐서들을 모델과 같은 위치(모델이 GPU니까 데이터도 GPU)로  옮기기
        X, y = X.to(device), y.to(device)
        
        # 예측(forward)
        pred = model(X) # softmax가 적용 안되어있음

        # Loss 계산
        loss = loss_fn(pred, y) # loss function 내에서 softmax가 적용 된 다음 y에 대한 loss를 구한다.

        # 역전파 수행
        optimizer.zero_grad() # 이전 배치에 남아있는 기울기를 제거
        loss.backward() # 오차 역전파
        optimizer.step()

        # 배치가 100번 돌 때마다 화면에 출력
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"Train Loss : {loss:>7f} [ {current:>5d} / {size:>5d} ]")

In [55]:
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)

    # loss는 배치 별로 계산, correct는 전체 데이터 세트에 대한 평균 정확도
    test_loss, correct = 0, 0

    # 모델을 추론 모드로 바꿔준다.
    model.eval()

    # 추론 과정에서는 기울기를 구할 필요가 없음. 따라서 모든 파라미터(model.parameters())의 required_grad=False
    with torch.no_grad():
        for X, y in dataloader:

            X, y = X.to(device), y.to(device)

            pred = model(X)

            # test_loss를 배치마다 구해서 더해주기
            test_loss += loss_fn(pred, y).item()

            # 10개의 예측 값중 가장 큰 곳의 인덱스를 argmax로 찾고, 타겟(y)와 일치하는지 확인
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    # 배치 개수 구하기
    num_batches = len(dataloader)

    # Loss 평균 구하기
    test_loss /= num_batches

    # Accuracy 구하기
    correct /= size

    print(f"Test Error : \n Accuracy : {(100*correct):>0.1f}%, Avg Loss : {test_loss:>8f}\n")

In [41]:
epochs = 10

for i in range(epochs):
    print(f"Epochs {i + 1}\n-------------------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
    print("\n")

print("Done!!!")

Epochs 1
-------------------------------------------
Train Loss : 0.429134 [     0 / 60000 ]
Train Loss : 0.532000 [  6400 / 60000 ]
Train Loss : 0.439087 [ 12800 / 60000 ]
Train Loss : 0.376231 [ 19200 / 60000 ]
Train Loss : 0.217501 [ 25600 / 60000 ]
Train Loss : 0.502792 [ 32000 / 60000 ]
Train Loss : 0.334343 [ 38400 / 60000 ]
Train Loss : 0.309730 [ 44800 / 60000 ]
Train Loss : 0.400801 [ 51200 / 60000 ]
Train Loss : 0.333813 [ 57600 / 60000 ]


Epochs 2
-------------------------------------------
Train Loss : 0.307991 [     0 / 60000 ]
Train Loss : 0.298345 [  6400 / 60000 ]
Train Loss : 0.414203 [ 12800 / 60000 ]
Train Loss : 0.280404 [ 19200 / 60000 ]
Train Loss : 0.341085 [ 25600 / 60000 ]
Train Loss : 0.444079 [ 32000 / 60000 ]
Train Loss : 0.405520 [ 38400 / 60000 ]
Train Loss : 0.426117 [ 44800 / 60000 ]
Train Loss : 0.363793 [ 51200 / 60000 ]
Train Loss : 0.382632 [ 57600 / 60000 ]


Epochs 3
-------------------------------------------
Train Loss : 0.287062 [     0 / 60000

여기까지가 01~~~~04. PyTorch Model Training | Validation까지 (240923)에 학습한 내용

# 훈련된 모델의 가중치를 저장 / 불러오기
불러올 곳에서 **모델의 구조를 알고 있는 경우** 가중치만 저장하면 적은 용량으로 저장하고 불러오는 것이 가능합니다.

In [58]:
# model.state_dict() : 모델 내에 있는 레이어 별 가중치를 들고 있는 딕셔너리
torch.save(model.state_dict(), 'model_weights.pth')

In [59]:
model.state_dict()

OrderedDict([('linear_relu_stack.0.weight',
              tensor([[-0.0110,  0.0362,  0.0385,  ..., -0.0009, -0.0659, -0.0108],
                      [ 0.0617, -0.0028,  0.0633,  ..., -0.0469, -0.0738, -0.0460],
                      [ 0.0419,  0.0107,  0.0445,  ...,  0.0703,  0.1044,  0.0528],
                      ...,
                      [ 0.0210,  0.0938,  0.0686,  ..., -0.0150, -0.0394,  0.0397],
                      [ 0.0263, -0.0276, -0.0540,  ..., -0.0514,  0.0012, -0.0247],
                      [-0.0313,  0.0123,  0.0644,  ..., -0.0210, -0.0018,  0.0526]],
                     device='mps:0')),
             ('linear_relu_stack.0.bias',
              tensor([ 2.4537e-01,  3.6434e-02, -8.9375e-02,  1.0470e-01,  3.5253e-01,
                       2.1932e-01,  5.5733e-02,  1.2115e-02, -2.2888e-01, -4.3831e-02,
                      -4.2541e-02,  1.3912e-01, -6.2271e-02, -8.9916e-03,  1.6770e-01,
                       5.2324e-02,  1.3337e-01, -9.7619e-03,  1.9156e-01, -1.0552e

In [60]:
# 저장된 가중치 파일(pth) 불러오기

# window
# model2 = NeuralNetwork().cuda() # .to('cuda')
# print(model2)

# mac (device = torch.device("mps"))
model2 = NeuralNetwork().to(device) # 모델을 MPS 또는 CPU로 이동
print(model2)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=10, bias=True)
  )
)


In [61]:
# 훈련되지 않은 모델로 검증하면 당연히 성능이 좋지 않다.
test_loop(test_dataloader, model2, loss_fn)

Test Error : 
 Accuracy : 10.6%, Avg Loss : 2.313483



In [62]:
# 이전에 훈련된 가중치를 불러와서 model2에서 로드
model2.load_state_dict(torch.load('model_weights.pth'))
test_loop(test_dataloader, model2, loss_fn)

/var/folders/zk/p59k4t_16mlgf68b5hxclv6m0000gn/T/ipykernel_31928/3017254588.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model2.load_state_dict(torch.load('model_weig

Test Error : 
 Accuracy : 87.2%, Avg Loss : 0.368632



# 훈련된 모델 자체를 저장 / 불러오기
모델의 구조를 모르는 경우 사용할 수 있는 대표적인 방법으로서 가중치만 저장한 경우보다 파일의 크기는 크지만 구조를 몰라도 모델을 사용할 수 있다는 장점이 있습니다.

In [63]:
torch.save(model, 'model.pth')

In [64]:
!ls -al | grep pth

-rw-r--r--   1 khb43  staff  410496 Sep 24 10:39 model.pth
-rw-r--r--   1 khb43  staff  409344 Sep 24 10:36 model_weights.pth


In [65]:
model3 = torch.load('model.pth')
test_loop(test_dataloader, model3, loss_fn)

/var/folders/zk/p59k4t_16mlgf68b5hxclv6m0000gn/T/ipykernel_31928/466480325.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model3 = torch.load('model.pth')


Test Error : 
 Accuracy : 87.2%, Avg Loss : 0.368632

